In [1]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
import numpy as np

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_columns', None)
pd.set_option('max_colwidth', 800)
pd.set_option('display.max_colwidth', None)

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [14]:
najpopularniejsi_artysci = [
    "Sanah",
    "Camila Cabello",
    "The Beatles",
    "Red Hot Chili Peppers",
    "Violetta Villas",
    "T Love",
    "Ewa Farna",
    "Cleo",
    "Margaret",
    "Sylwia Przybysz",
    "Maryla Rodowicz",
    "Czeslaw Niemen",
    "Maciej Malenczuk",
    "Dzem",
    "Kult",
    "Budka Suflera",
    "Marek Grechuta",
    "Republika",
    "Edyta Bartosiewicz",
    "Wilki",
    "Varius Manx",
    "Peja",
    "Kizo",
    "Paktofonika",
    "Baciary",
    "Weekend",
    "Boys",
    "Pezet",
    "Natalia Szroeder",
    "Kamil Bednarek",
    "Roksana Wegiel",
    "Mata",
    "Sobel",
    "Sokol",
    "Taco Hemingway",
    "Young Multi",
    "Daria Zawialow"
    

]
new = []
print(len(najpopularniejsi_artysci))
for i in najpopularniejsi_artysci:
    new.append(i.replace(" ","_").lower())
    #break

37


In [16]:
new

['sanah',
 'camila_cabello',
 'the_beatles',
 'red_hot_chili_peppers',
 'violetta_villas',
 't_love',
 'ewa_farna',
 'cleo',
 'margaret',
 'sylwia_przybysz',
 'maryla_rodowicz',
 'czeslaw_niemen',
 'maciej_malenczuk',
 'dzem',
 'kult',
 'budka_suflera',
 'marek_grechuta',
 'republika',
 'edyta_bartosiewicz',
 'wilki',
 'varius_manx',
 'peja',
 'kizo',
 'paktofonika',
 'baciary',
 'weekend',
 'boys',
 'pezet',
 'natalia_szroeder',
 'kamil_bednarek',
 'roksana_wegiel',
 'mata',
 'sobel',
 'sokol',
 'taco_hemingway',
 'young_multi',
 'daria_zawialow']

### New

In [17]:
def data_scrap(new):

    df = None
    part_link = 'https://www.tekstowo.pl'
    df = pd.DataFrame(data = {"Link":[]})
    for i in new:
        artist_link = f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,1.html"

        print(artist_link)
        
        page = requests.get(artist_link)
        soup = BeautifulSoup(page.text, "lxml")

        max_num_page = soup.find_all("a", class_ = "page-link")[-2].text
        max_num_page = int(max_num_page)
        # print(max_num_page)
        
        
        for num_page in range(1,max_num_page+1):
            num_page = int(num_page)
            # print(num_page)
            boxes = soup.find_all("div", class_ = "flex-group")
            
            for box in boxes:
                try:
                    #label = box.find("div",class_ = "flex-group").text
                    link = box.find("a",class_ = "title").get("href")
                    song_link = part_link + link
                    #print(song_link)
                    df = df.append({"Link": song_link}, ignore_index = True)
                    
                    #df = df_text.append({"Text":box}, ignore_index = True)   
                except:
                    pass
            
            next_number_page = num_page + 1
            next_page = f"https://www.tekstowo.pl/piosenki_artysty,{i},alfabetycznie,strona,{next_number_page}.html"
            # print(next_page)
            page = requests.get(next_page)
            soup = BeautifulSoup(page.text, "lxml")

              
    return df   

df = data_scrap(new)


https://www.tekstowo.pl/piosenki_artysty,sanah,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,camila_cabello,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,the_beatles,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,red_hot_chili_peppers,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,violetta_villas,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,t_love,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,ewa_farna,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,cleo,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,margaret,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,sylwia_przybysz,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,maryla_rodowicz,alfabetycznie,strona,1.html
https://www.tekstowo.pl/piosenki_artysty,czeslaw_niemen,alfabetycznie,strona,1.html
https://www.tekstowo.pl

In [9]:
df.head(13)

,Link
0,"https://www.tekstowo.pl/piosenka,sanah,_ii__da_bog_kiedys_zasiasc_w_polsce_wolnej__a__asnyk_.html"
1,"https://www.tekstowo.pl/piosenka,sanah,_i__da_bog_kiedys_zasiasc_w_polsce_wolnej__a__asnyk_.html"
2,"https://www.tekstowo.pl/piosenka,sanah,_hot16challenge2.html"
3,"https://www.tekstowo.pl/piosenka,sanah,2_00.html"
4,"https://www.tekstowo.pl/piosenka,sanah,2_10.html"
5,"https://www.tekstowo.pl/piosenka,sanah,ale_jazz___na_jazzowo.html"
6,"https://www.tekstowo.pl/piosenka,sanah,ale_jazz_feat__vito_bambino.html"
7,"https://www.tekstowo.pl/piosenka,sanah,alternatywka.html"
8,"https://www.tekstowo.pl/piosenka,sanah,aniolom_szepnij_to.html"
9,"https://www.tekstowo.pl/piosenka,sanah,audi__feat__mietha_.html"


In [6]:
def text_scrap(df):
    df_text = None
    df_text = pd.DataFrame(data = {"Polish":[], 
                                   "English": []})

    for index, row in df.iterrows():
        try:
            
            
            page = requests.get(row[0])
            #print(page)
            soup = BeautifulSoup(page.text, "lxml")

            original = soup.find_all("div", class_ = "inner-text")[0].text
            translated = soup.find_all("div", class_ = "inner-text")[1].text

            translated = np.nan if translated == "" else translated
            new_data = None
            df_temp = None
            if re.search(r'[ąćęłńóśźżĄĆĘŁŃÓŚŹŻ]', original):

                # df_text = df_text.append({"Polish":original, "English": translated}, ignore_index = True)
                new_data = {"Polish": [original], "English": [translated]}
                df_temp = pd.DataFrame(data = new_data)
                df_text = pd.concat([df_text, df_temp], ignore_index = True)
            else:

                # df_text = df_text.append({"Polish":translated, "English":original}, ignore_index = True)
                new_data = {"Polish":[translated], "English":[original]}
                df_temp = pd.DataFrame(data = new_data)
                df_text = pd.concat([df_text, df_temp], ignore_index = True)
        except:
            pass
     
    return df_text

            
df_text = text_scrap(df)

In [10]:
# df_text

In [6]:
print(len(df_text))
print(len(df))
df_text.iloc[[1896]]

0
184


IndexError: positional indexers are out-of-bounds

In [39]:
df_text.to_csv("Dataset2.csv", index = False, encoding = "utf-8")

In [2]:
df_text = pd.read_csv("H:\AI_Song_Writer\Dataset2.csv")

In [11]:
# df_text.head(20)

In [14]:
df_text["English"].isna().sum()

316

In [13]:
# 400 wiersz - polski i angielski razem (dla pierwszych dwoch z listy new)
# df_text.iloc[[1509]]

In [12]:
# df_text.iloc[[799]]

In [5]:
df.duplicated().sum()

0

In [6]:
df[df["Link"] == "https://www.tekstowo.pl/piosenka,taylor_swift,dorothea.html"]

,Link
79,"https://www.tekstowo.pl/piosenka,taylor_swift,dorothea.html"
